In [2]:
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import RMSprop, Adam
import numpy as np
from keras.utils import np_utils
from keras import regularizers
import os

Using TensorFlow backend.


In [3]:
batch_size = 128 # batch 的大小，如果出現 OOM error，請降低這個值
num_classes = 10 # 類別的數量，Cifar 10 共有 10 個類別
epochs = 10 # 訓練的 epochs 數量

# 讀取資料並檢視
(x_train, y_train), (x_test, y_test) = cifar10.load_data()


# 對 label 進行 one-hot encoding (y_trian 原本是純數字)
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
 
#z-score
mean = np.mean(x_train,axis=(0,1,2,3))
std = np.std(x_train,axis=(0,1,2,3))
x_train = (x_train-mean)/(std+1e-7)
x_test = (x_test-mean)/(std+1e-7)
 
num_classes = 10
y_train = np_utils.to_categorical(y_train,num_classes)
y_test = np_utils.to_categorical(y_test,num_classes)
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
print('y_train shape:', y_train.shape)
print(y_train.shape[0], 'train samples')
print(y_test.shape[0], 'test samples')

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples
y_train shape: (50000, 10)
50000 train samples
10000 test samples


## 接下來我們使用 CNN 來訓練神經網路
CNN 的原理非常適合處理影像類的資料，就讓我們來看看，同樣的訓練條件，CNN 是否顯著優於 DNN 呢?

In [4]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [5]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))
model.summary()

W0809 22:42:57.741190 4567180736 deprecation_wrapper.py:119] From /Users/vincent/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0809 22:42:57.763464 4567180736 deprecation_wrapper.py:119] From /Users/vincent/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0809 22:42:57.767935 4567180736 deprecation_wrapper.py:119] From /Users/vincent/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0809 22:42:57.793318 4567180736 deprecation_wrapper.py:119] From /Users/vincent/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0809 22:42:57.7951

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 30, 30, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 30, 30, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 15, 15, 64)        18496     
__________

In [9]:

#data augmentation

datagen = ImageDataGenerator(rotation_range=30, width_shift_range=0.1,
                         height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,
                         horizontal_flip=True, fill_mode="nearest")
datagen.fit(x_train)
 

model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),\
                    steps_per_epoch=x_train.shape[0] // batch_size,epochs=epochs,\
                    verbose=1,validation_data=(x_test,y_test))

Epoch 1/10
390/390 [==============================] - 71s 181ms/step - loss: 1.1197 - acc: 0.6277 - val_loss: 1.0137 - val_acc: 0.6868
Epoch 2/10
390/390 [==============================] - 75s 191ms/step - loss: 1.0718 - acc: 0.6405 - val_loss: 0.7350 - val_acc: 0.7565
Epoch 3/10
390/390 [==============================] - 79s 203ms/step - loss: 1.0612 - acc: 0.6431 - val_loss: 0.7838 - val_acc: 0.7424
Epoch 4/10
390/390 [==============================] - 81s 209ms/step - loss: 1.0441 - acc: 0.6500 - val_loss: 0.7918 - val_acc: 0.7407
Epoch 5/10
390/390 [==============================] - 84s 214ms/step - loss: 1.0465 - acc: 0.6476 - val_loss: 0.8107 - val_acc: 0.7421
Epoch 6/10
390/390 [==============================] - 85s 219ms/step - loss: 1.0345 - acc: 0.6517 - val_loss: 0.8786 - val_acc: 0.7185
Epoch 7/10
390/390 [==============================] - 86s 221ms/step - loss: 1.0340 - acc: 0.6493 - val_loss: 0.7395 - val_acc: 0.7597
Epoch 8/10
390/390 [==============================] - 8

In [10]:

score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.9136159287452698
Test accuracy: 0.7142


In [8]:
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 50000 samples, validate on 10000 samples
Epoch 1/10
50000/50000 [==============================] - 71s 1ms/step - loss: 0.7716 - acc: 0.7354 - val_loss: 0.8027 - val_acc: 0.7242
Epoch 2/10
50000/50000 [==============================] - 71s 1ms/step - loss: 0.7285 - acc: 0.7511 - val_loss: 0.6711 - val_acc: 0.7736
Epoch 3/10
50000/50000 [==============================] - 73s 1ms/step - loss: 0.7056 - acc: 0.7588 - val_loss: 0.6525 - val_acc: 0.7816
Epoch 4/10
50000/50000 [==============================] - 74s 1ms/step - loss: 0.6802 - acc: 0.7692 - val_loss: 0.7555 - val_acc: 0.7651
Epoch 5/10
50000/50000 [==============================] - 74s 1ms/step - loss: 0.6755 - acc: 0.7722 - val_loss: 0.6531 - val_acc: 0.7889
Epoch 6/10
50000/50000 [==============================] - 75s 1ms/step - loss: 0.6568 - acc: 0.7772 - val_loss: 0.7573 - val_acc: 0.7785
Epoch 7/10
50000/50000 [==============================] - 75s 1ms/step - loss: 0.6489 - acc: 0.7831 - val_loss: 0.8390 - val_acc